# Feature

## Setup

In [ ]:
#| default_exp data.feature

In [ ]:
#| export
# basics
import pandas as pd, numpy as np
from tqdm import tqdm
import hashlib,numba
from tqdm.contrib.concurrent import process_map

from skfp.fingerprints import AtomPairFingerprint, ECFPFingerprint, MACCSFingerprint
from kdock.data.core import Data

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # suppress umap tensorflow warning
from umap import UMAP  # import UMAP class properly

from rdkit import Chem
from rdkit.Chem import Descriptors,Descriptors3D, AllChem

## Rdkit feature

In [ ]:
#| export
def get_rdkit(SMILES:str):
    """
    Extract chemical features from SMILES
    Reference: https://greglandrum.github.io/rdkit-blog/posts/2022-12-23-descriptor-tutorial.html
    """
    mol = Chem.MolFromSmiles(SMILES)
    return Descriptors.CalcMolDescriptors(mol)

In [ ]:
#| export
def get_rdkit_3d(SMILES:str):
    "Extract 3d features from SMILES"
    mol = Chem.MolFromSmiles(SMILES)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol, AllChem.ETKDG())
    AllChem.UFFOptimizeMolecule(mol)
    return Descriptors3D.CalcMolDescriptors3D(mol)

In [ ]:
#| export
def get_rdkit_all(SMILES:str):
    "Extract chemical features and 3d features from SMILES"
    feat = get_rdkit(SMILES)
    feat_3d = get_rdkit_3d(SMILES)
    return feat|feat_3d

In [ ]:
#| export
def remove_hi_corr(df: pd.DataFrame, 
                   thr=0.99 # threshold
                   ):
    "Remove highly correlated features in a dataframe given a pearson threshold"
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > thr)]
    return df.drop(to_drop, axis=1), to_drop

In [ ]:
#| export
def preprocess(df: pd.DataFrame, thr=0.99):
    "Remove features with no variance, and highly correlated features based on threshold."
    col_ori = df.columns

    # Remove columns with std == 0
    std_zero_cols = df.columns[df.std() == 0].tolist()
    
    if std_zero_cols:
        n=len(std_zero_cols)
        print(f"\n {n} Columns with zero std: {std_zero_cols}")
    df = df.loc[:, df.std() != 0].copy()

    # Remove highly correlated columns
    df, high_corr_cols = remove_hi_corr(df, thr)
    if high_corr_cols:
        n=len(high_corr_cols)
        print(f"\n {n} Columns removed due to high similarity (pearson>{thr}): {high_corr_cols}")

    dropping_col = set(col_ori) - set(df.columns)
    n = len(dropping_col)
    print(f"\n Total removed columns: {n}")
    
    return df

In [ ]:
#| export
def get_rdkit_df(df: pd.DataFrame,
                 include_3d=False,
                 col='SMILES',
                 postprocess=False,
                 chunksize=128, # for parallel process_map 
                 ):
    "Extract rdkit features (optionally in parallel with 3D) from SMILES in a df"
    func = get_rdkit_all if include_3d else get_rdkit
    smiles_list = df[col].tolist()

    features = process_map(func, smiles_list,chunksize=chunksize)

    out = pd.DataFrame(features)

    if postprocess:
        out = StandardScaler().fit_transform(out)
        out = preprocess(out) # remove redundant

    return out

In [ ]:
# df=Data.collins.get_antibiotics_2k().head(300)
# df.head()

In [ ]:
# %%time
# feat_raw=get_rdkit_df(df)
# feat_raw

In [ ]:
# feat = get_rdkit_df(df,postprocess=True)
# feat

## Morgan fingerprints (ECFP)

Reference:
- [kaggle notebook, scikit-fingerprints](https://www.kaggle.com/code/michaszafarczyk/molecular-fingerprints-using-scikit-fingerprints)
- [scikit-fingerprints github](https://github.com/scikit-fingerprints/scikit-fingerprints)
- [kaggle notebook, fingerprint tips and tricks](https://www.kaggle.com/code/towardsentropy/fingerprint-tips-and-tricks)

In [ ]:
#| export
def get_fp(SMILES, 
           name='ecfp',
           ELEMENTS_PER_WORKER = 1_000_000):
    "Super fast method to get molecule fingerprints using scikit-fingerprints"
    if name == 'ecfp':
        fp_transformer = ECFPFingerprint(fp_size=2048, radius=3, n_jobs=-1)
    elif name == 'ap':
        fp_transformer = AtomPairFingerprint(fp_size=2048, n_jobs=-1)
    elif name == 'maccs':
        fp_transformer = MACCSFingerprint(n_jobs=-1)
    else:
        raise Exception('Wrong fingerprint name!')
    
    middle_parts = []
    k_splits = len(SMILES) // ELEMENTS_PER_WORKER
        
    for i in tqdm(range(k_splits)):
        middle_parts.append(fp_transformer.transform(SMILES[i * ELEMENTS_PER_WORKER: (i + 1) * ELEMENTS_PER_WORKER]))
        
    if len(SMILES) % ELEMENTS_PER_WORKER > 0:   
        middle_parts.append(fp_transformer.transform(SMILES[k_splits * ELEMENTS_PER_WORKER:]))
    
    return np.concatenate(middle_parts)

In [ ]:
# %%time
# fp = get_fp(df.SMILES, name='ecfp')
# fp

## Use np.packbits to save space
> Save space when dealing with super big dataset

Packbits will transform binary to decimals, if it is [1, 0, 1, 1, 0, 0, 0, 0, 1, 1], then it will take 8 bits in the sequence, [10110000] and [11000000], for the latter, if not enough for 8 bits length, it will add 0. 

To transform binary to decimals, take [10110000] for example, calculate through 1*2^7 + 0*2^6 + 1*2^5 + 1*2^4 + 0*2^3 + 0*2^2 + 0*2^1 + 0*2^0=176

In [ ]:
#| export
def compress_fp(array):
    "Compress binary finterprints using np.packbits"
    return np.packbits(array,axis=1)

In [ ]:
# compress_fp(fp)

## Tanimoto

Reference: https://www.kaggle.com/code/towardsentropy/fingerprint-tips-and-tricks/notebook

Slow version:

Method 1

In [ ]:
from scipy.spatial import distance

In [ ]:
# %%time
# tanimotos_scipy = 1-distance.cdist(fp, fp, metric='jaccard')
# # 30.4s

A little faster with parallel

In [ ]:
from sklearn.metrics import pairwise_distances

In [ ]:
# %%time
# tanimotos_sklearn = 1 - pairwise_distances(fp, metric='jaccard', n_jobs=-1)
# # 13.1s

Method 2

In [ ]:
from rdkit.Chem import DataStructs

In [ ]:
# %%time
# tanimotos_rdkit = np.array([DataStructs.BulkTanimotoSimilarity(i, fp) for i in fp])

Method 3

In [ ]:
def tanimoto_bitwise(fps_np):
    intersection = (fps_np[:,None] & fps_np[None]).sum(-1)
    union = (fps_np[:,None] | fps_np[None]).sum(-1)
    return intersection / union

In [ ]:
# tanimoto_bitwise(fp)

### Numba tanimoto

In [ ]:
#| export
@numba.njit(parallel=True)
def tanimoto_numba(fps):
    "Get a NxN matrix of tanimoto similarity among N compounds."
    n = fps.shape[0]
    result = np.zeros((n, n), dtype=np.float32)
    
    for i in numba.prange(n):
        for j in range(i, n):  # only upper triangle
            inter = np.bitwise_and(fps[i], fps[j]).sum()
            union = np.bitwise_or(fps[i], fps[j]).sum()
            sim = inter / union if union > 0 else 0.0
            result[i, j] = sim
            result[j, i] = sim  # fill symmetric position
    return result

In [ ]:
# %%time
# tanimoto_numba(fp)

## Group same compounds
> Utilize hash sha256 to encode morgan fp and find same molecule

In [ ]:
#| export
def hash_fp(fp_row):
    "Hash a binary fingerprint row using SHA256"
    return hashlib.sha256(fp_row.tobytes()).hexdigest()

In [ ]:
# hash_fp(fp[0])

In [ ]:
#| export
def get_same_mol_group(df, smi_col='SMILES'):
    "Assign a group number to the same compounds by utilizing hash sha256 to encode morgan fp and find same molecule."
    df = df.copy()
    smiles = df[smi_col].tolist()
    
    fps = get_fp(smiles)
    
    # Hash each fingerprint
    # hashes = [hash_fp(fp) for fp in fps] # non-parallel
    hashes = process_map(hash_fp, fps, chunksize=256) # if parallel
    
    # Assign a group number based on hash buckets
    hash_to_group = {}
    group_ids = []
    group_counter = 0
    for h in hashes:
        if h not in hash_to_group:
            hash_to_group[h] = group_counter
            group_counter += 1
        group_ids.append(hash_to_group[h])
    
    df['group'] = group_ids
    return df

In [ ]:
# %%time
# df2 = get_same_mol_group(df)
# df2.group.value_counts()

## Dimensionality reduction

In [ ]:
#| export
def reduce_feature(data, # df or numpy array
                   method='pca', # dimensionality reduction method, accept both capital and lower case
                   complexity=20, # None for PCA; perfplexity for TSNE, recommend: 30; n_neigbors for UMAP, recommend: 15
                   n=2, # n_components
                   seed: int=123, # seed for random_state
                   **kwargs, # arguments from PCA, TSNE, or UMAP depends on which method to use
                  ):
    
    "Reduce the dimensionality given a dataframe of values"
    
    method = method.lower()
    assert method in ['pca','tsne','umap'], "Please choose a method among PCA, TSNE, and UMAP"

    if method == 'pca':
        reducer = PCA(n_components=n, random_state=seed,**kwargs)
    elif method == 'tsne':
        reducer = TSNE(n_components=n,
                       random_state=seed, 
                       perplexity = complexity, # default from official is 30 
                      **kwargs)
    elif method == 'umap':
        reducer = UMAP(n_components=n, 
                       random_state=seed, 
                       n_neighbors=complexity, # default from official is 15, try 15-200
                      **kwargs)
    else:
        raise ValueError('Invalid method specified')

    proj = reducer.fit_transform(data)
    embedding_df = pd.DataFrame(proj).set_index(data.index) if isinstance(data,pd.DataFrame) else pd.DataFrame(proj)
    embedding_df.columns = [f"{method.upper()}{i}" for i in range(1, n + 1)]

    return embedding_df

In [ ]:
# pca = reduce_feature(fp,'pca',n=10)
# pca

## End

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

## Old

In [ ]:
#| hide
# #| export
# def tanimoto(df, # df with SMILES and ID columns
#              smiles_col='SMILES', # colname of SMILES
#              id_col='ID', # colname of compound ID
#              target_col=None, # colname of compound values (e.g., IC50)
#              radius=2, # radius of the Morgan fingerprint.
#              ):
#     "Calculates the Tanimoto similarity scores between all pairs of molecules in a pandas DataFrame."
    
#     df = df.copy()
#     # Convert SMILES to molecule objects
#     df['Molecule'] = df[smiles_col].apply(lambda x: Chem.MolFromSmiles(x))

#     # Calculate fingerprints
#     df['Fingerprint'] = df['Molecule'].apply(lambda x: AllChem.GetMorganFingerprintAsBitVect(x, radius))

#     # Calculate similarity scores
#     similarity_scores = []
#     for i in range(len(df)):
#         for j in range(i+1, len(df)):
#             sim_score = DataStructs.TanimotoSimilarity(df['Fingerprint'][i], df['Fingerprint'][j])
#             if target_col is not None:
#                 similarity_scores.append((df[id_col][i], df[id_col][j], df[smiles_col][i], df[smiles_col][j], sim_score, df[target_col][i], df[target_col][j]))
#             else:
#                 similarity_scores.append((df[id_col][i], df[id_col][j], df[smiles_col][i], df[smiles_col][j], sim_score))

#     # Create a new DataFrame with the similarity scores
#     if target_col is not None:
#         result_df = pd.DataFrame(similarity_scores, columns=['ID1', 'ID2', 'SMILES1', 'SMILES2', 'SimilarityScore', 'Target1', 'Target2'])
#     else:
#         result_df = pd.DataFrame(similarity_scores, columns=['ID1', 'ID2', 'SMILES1', 'SMILES2', 'SimilarityScore'])

#     # Sort by similarity score in descending order
#     result_df.sort_values('SimilarityScore', ascending=False, inplace=True)
#     result_df = result_df.reset_index(drop=True)

#     return result_df

# df = Data.get_mirati_g12d_raw()[['ID','SMILES','IC50']]
# df = df.dropna(subset= 'IC50').reset_index(drop=True)

# df.head()

# # result = tanimoto(df.head(), target_col = 'IC50')